In [1]:
# model의 setting을 저장한 yaml 파일 확인
import os
os.listdir('../experiments/cityscapes')

['.ipynb_checkpoints',
 'seg_hrnet_w48_trainval_ohem_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484x2.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml',
 'seg_hrnet_ocr_w48_trainval_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484_paddle.yaml',
 'seg_hrnet_w48_train_ohem_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle_custom.yaml',
 'seg_hrnet_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml',
 'seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484.yaml',
 'seg_hrnet_w48_trainval_512x1024_sgd_lr1e-2_wd5e-4_bs_12_epoch484x2.yaml']

In [2]:
# 모델 load test
from models.seg_hrnet_ocr import get_seg_model
get_seg_model()

# config 파일을 필요로함 

TypeError: get_seg_model() missing 1 required positional argument: 'cfg'

In [3]:
# form import 문처럼 config를 사용이 가능함  
# 실제로 config load를 하면 model(seg HRNet)과 config의 내용이 맞지 않음 
from config import config
get_seg_model(config)

KeyError: 'STAGE1'

In [3]:
# cofing 파일을 보면 기존의 config을 생성후에 이를 파일에서 읽어서 하는 구조임
# updata_conifg 함수를 수정해서 사용하도록 만듬

import yaml
from config import config
from yacs.config import CfgNode as CN
from models.seg_hrnet_ocr import get_seg_model


def update_config(cfg, cfg_file_path):
    cfg.defrost()

    # 파일에서 YAML 설정 읽기
    with open(cfg_file_path, 'r') as f:
        yaml_cfg = yaml.safe_load(f)
    
    yaml_cfg['MODEL']['PRETRAINED'] = '../model_ckpt/hrnet_ocr_cs_trainval_8227_torch11.pth'
    #yaml_cfg['MODEL']['NUM_OUTPUTS'] = 3
    cfg.merge_from_other_cfg(CN(yaml_cfg))
    cfg.freeze()

In [5]:
import os
import torch
import numpy as np
import segmentation_models_pytorch as smp
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader


class SkinDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_dir = os.path.join(root_dir, 'images')
        self.mask_dir = os.path.join(root_dir, 'masks')
        
        # Add a filter to exclude directories
        self.image_files = sorted([f for f in os.listdir(self.image_dir) if os.path.isfile(os.path.join(self.image_dir, f))])
        self.mask_files = sorted([f for f in os.listdir(self.mask_dir) if os.path.isfile(os.path.join(self.mask_dir, f))])
        
        self.transform = transform
        
    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_files[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        unique_values = np.unique(mask)
        image = self.transform(image)

        mask_np = np.array(mask)
        masks = [(mask_np == val).astype(int) for val in range(3)]
        mask = np.stack(masks, axis=0)
        mask = torch.tensor(mask, dtype=torch.float32)
        return image, mask


In [6]:
batch_size = 2
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor()
])
train_data = SkinDataset(root_dir='../../../../../Datasets/NewSkinDataset/original/train', transform=transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = SkinDataset(root_dir='../../../../../Datasets/NewSkinDataset/original/val', transform=transform)
test_loader = DataLoader(test_data, batch_size=10, shuffle=True)


In [7]:
update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
model = get_seg_model(config)

set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


In [8]:
for batch in test_loader:
    image , mask = batch
    result = model(image)
    print(len(result) , type(result) , result[0].shape)
    break

2 <class 'list'> torch.Size([10, 19, 160, 160])


In [10]:
# config 파일에서 num_class를 변경하면 다음과 같은 에러가 뜸
update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle_custom.yaml')
model = get_seg_model(config)

set()
{'loss.criterion1.weight', 'loss.criterion0.weight'}


RuntimeError: Error(s) in loading state_dict for HighResolutionNet:
	size mismatch for cls_head.weight: copying a param with shape torch.Size([19, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([3, 512, 1, 1]).
	size mismatch for cls_head.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([3]).
	size mismatch for aux_head.3.weight: copying a param with shape torch.Size([19, 720, 1, 1]) from checkpoint, the shape in current model is torch.Size([3, 720, 1, 1]).
	size mismatch for aux_head.3.bias: copying a param with shape torch.Size([19]) from checkpoint, the shape in current model is torch.Size([3]).

* dataset의 num_class를 변경 하면 위와 같은에러가 뜸  

기존의 모델 생성 후에 classification layer를 추가해서 [batch, num_class(3) , width, height]로 변경을 해야 하나?  

* output의 width , height의 크기가 일치하지 않음  
Unet 구조가 아님  


In [8]:
# 모델으 학습을 진행했을 때 에러 코드 
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
import segmentation_models_pytorch as smp

from torch import nn
#from datasets import load_metric
from torchvision import transforms
from PIL import Image
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint


class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, ignore_index=0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = F.cross_entropy(inputs, targets, reduction='none', ignore_index=self.ignore_index)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss
        

class LitEfficientNet(pl.LightningModule):
    def __init__(self, train_dataloader=None, val_dataloader=None, test_dataloader=None, batch_size=32, config=None):
        super(LitEfficientNet, self).__init__()
        self.train_data_loader = train_dataloader
        self.val_data_loader = val_dataloader
        self.test_data_loader = test_dataloader
        
        from config import config
        update_config(config, '../experiments/cityscapes/seg_hrnet_ocr_w48_train_512x1024_sgd_lr1e-2_wd5e-4_bs_16_epoch484_paddle.yaml')
        self.model = get_seg_model(config)
        
        self.criterion = FocalLoss(alpha=1, gamma=2, ignore_index=0)
        #$self.train_mean_iou = load_metric("mean_iou")
        #self.val_mean_iou = load_metric("mean_iou")
        #self.test_mean_iou = load_metric("mean_iou")
        self.batch_size = batch_size

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.argmax(dim=1)
        preds = self(x)
        loss = self.criterion(preds, y)

        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        #y = y.argmax(dim=1)
        preds = self(x)
        print(preds[0].shape)
        print(preds[1].shape)
        print(preds[2].shape)
        loss = self.criterion(preds, y)
        
        self.log('val_loss', loss)
        return {'val_loss': loss}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.argmax(dim=1)
        preds = self(x)
        loss = self.criterion(preds, y)

        return {'test_loss': loss}

    
    def configure_optimizers(self):
        return torch.optim.Adam([p for p in self.parameters() if p.requires_grad], lr=1e-08, eps=1e-08)
    
    def train_dataloader(self):
        return self.train_data_loader
    
    def val_dataloader(self):
        return self.val_data_loader
    
    def test_dataloader(self):
        return self.test_data_loader


In [9]:
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val_loss")
early_stop_callback = EarlyStopping(
    monitor="val_loss", 
    min_delta=0.00, 
    patience=3, 
    verbose=False, 
    mode="min",
)

model = LitEfficientNet(train_dataloader=train_loader ,
                        val_dataloader=test_loader,
                        test_dataloader=test_loader,
                        )

pl.seed_everything(42)
trainer = pl.Trainer(max_epochs=500,
                     callbacks=[early_stop_callback, checkpoint_callback],
                     gpus=[1])
trainer.fit(model)
trainer.save_checkpoint('./model_pt/2023_09_13_focal_loss.ckpt')

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


set()
{'loss.criterion0.weight', 'loss.criterion1.weight'}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name      | Type              | Params
------------------------------------------------
0 | model     | HighResolutionNet | 70.4 M
1 | criterion | FocalLoss         | 0     
------------------------------------------------
70.4 M    Trainable params
0         Non-trainable params
70.4 M    Total params
281.491   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

torch.Size([10, 19, 160, 160])
torch.Size([10, 19, 160, 160])


IndexError: list index out of range